In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
import tensorflow as tf
import numpy as np
from tqdm import tqdm
from sklearn.decomposition import PCA
import os
from sklearn.model_selection import GridSearchCV

In [ ]:
datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
)
test_datagen = ImageDataGenerator(rescale=1.0/255)
notebook_dir = os.getcwd()
project_root = os.path.dirname(notebook_dir)

In [ ]:
train_dir = os.path.join(project_root, 'Image_processing', 'DATASET', 'TRAIN')
test_dir = os.path.join(project_root, 'Image_processing', 'DATASET', 'TEST')

In [ ]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(331, 331),
    batch_size=32,
    class_mode='binary',
    shuffle=True
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(331, 331),
    class_mode='binary',
    shuffle=False
)

In [ ]:
model_new = load_model('nasnet-model.h5')

In [ ]:
model_feat = tf.keras.Model(inputs=model_new.input,outputs=model_new.layers[-2].output)

feat_train = model_feat.predict(train_generator)

In [ ]:
pca = PCA(n_components=100)
feat_train_pca = pca.fit_transform(feat_train)

In [ ]:
feat_test = model_feat.predict(test_generator)
feat_test_pca = pca.fit_transform(feat_test)

In [ ]:
labels = []
for i in tqdm(range(565)):
    inputs, label_batch = next(train_generator)
    labels.extend(label_batch)
labels = np.array(labels)
labels.shape

In [ ]:
y_test = []
for i in tqdm(range(79)):
    inputs, label_batch = next(test_generator)
    y_test.extend(label_batch)
y_test = np.array(y_test)
y_test.shape

In [ ]:
param_grid = {
    'C': [0.0625, 0.125, 0.25, 0.5, 1, 2, 4, 8, 16],
    'gamma': [0.015625,0.03125, 0.0625, 0.125, 0.25, 0.5, 1]
}
grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(feat_train_pca, labels)